<a href="https://colab.research.google.com/github/mohamedabk/NikeEmbeddingAssignment/blob/main/Machine_learning_based_methods_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pickle

uploaded = files.upload()

In [ ]:
import pandas as pd

df = pd.read_csv('tweets-data.csv')
df.head()  # Pour vérifier les colonnes

In [ ]:
import re
import nltk
nltk.download('punkt', force=True)
nltk.download('stopwords', force=True)

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def clean_tweet_v2(text):
    text = str(text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    tokens = text.split()  # Pas de word_tokenize
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)


In [ ]:
df['cleaned_text_v2'] = df['Tweets'].apply(clean_tweet_v2)
df[['Tweets', 'cleaned_text_v2']].head()

In [ ]:
# Installer la bibliothèque Transformers
!pip install transformers --quiet

# Importer le pipeline
from transformers import pipeline

# Charger le pipeline de sentiment (modèle par défaut)
sentiment_pipeline = pipeline("sentiment-analysis")

# Fonction pour appliquer le pipeline et éviter les erreurs de texte trop long
def get_transformer_sentiment(text):
    text = str(text)
    # Tronquer le texte s’il dépasse 1000 caractères (limite BERT ≈ 512 tokens)
    if len(text) > 1000:
        text = text[:1000]
    result = sentiment_pipeline(text)[0]
    return result['label'].lower(), result['score']

# Appliquer la fonction sur la colonne cleaned_text_v2
df[['transformer_sentiment', 'transformer_score']] = df['cleaned_text_v2'].apply(
    lambda x: pd.Series(get_transformer_sentiment(x))
)

# Afficher les premières lignes
df[['cleaned_text_v2', 'transformer_sentiment', 'transformer_score']].head()

